BiLSTM+CRF模型训练，使用bert作为embed嵌入，bert冻结不更新

In [1]:
# 显卡查看
! nvidia-smi

Thu May 21 01:54:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# 依赖安装
! pip install fastNLP

加载数据集

In [3]:
import sys

from fastNLP.core import Const
from fastNLP.io import PeopleDailyNERLoader
from fastNLP.io import PeopleDailyPipe

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [0]:
# 原始语料
# train_file_config = {
#     'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train.conll',
#     'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev.conll',
#     'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
# }

In [0]:
# 使用最佳模型预测后的语料
train_file_config = {
    'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_bert_predict.conll',
    'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev_bert_predict.conll',
    'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
}

In [6]:
# train_data_bundle_pkl_file = './data/weibo_NER/train_data_bundle.pkl'
logger.info('数据加载')
data_loader = PeopleDailyNERLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = PeopleDailyPipe()
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)
# save_serialize_obj(data_bundle, train_data_bundle_pkl_file)
# logger.info('数据预处理后进行序列化：{}'.format(train_data_bundle_pkl_file))

2020-05-21 01:55:01 I [<ipython-input-6-a24be1271d63>:2] 数据加载
2020-05-21 01:55:01 I [utils.py:16] dataset name : train
2020-05-21 01:55:01 I [utils.py:17] dataset len : 1350
2020-05-21 01:55:01 I [utils.py:18] dataset example : 
2020-05-21 01:55:01 I [utils.py:19] 
+------------------------------------------+------------------------------------------+
| raw_chars                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['浑', '身', '酸', '疼', '，', '两', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+----------------

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-05-21 01:55:02 I [utils.py:16] dataset name : train
2020-05-21 01:55:02 I [utils.py:17] dataset len : 1350
2020-05-21 01:55:02 I [utils.py:18] dataset example : 
2020-05-21 01:55:02 I [utils.py:19] 
+------------------------+------------------------+---------+------------------------+
| raw_chars              | target                 | seq_len | words                  |
+------------------------+------------------------+---------+------------------------+
| ['科', '技', '全', ... | [0, 0, 0, 0, 0, 0, ... | 26      | [792, 1015, 156, 19... |
| ['对', '，', '输', ... | [0, 0, 0, 0, 0, 0, ... | 15      | [123, 2, 1205, 115,... |
| ['今', '天', '下', ... | [0, 0, 0, 0, 0, 0, ... | 79      | [164, 33, 65, 739, ... |
| ['今', '年', '拜', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [164, 92, 880, 92, ... |
| ['浑', '身', '酸', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [2072, 221, 1056, 9... |
+------------------------+------------------------+---------+------------------------+
2020-05-21 01:55:02 I [utils

+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+---------

模型训练

In [0]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.embeddings import BertEmbedding
from fastNLP.models import BiLSTMCRF
from fastNLP import SpanFPreRecMetric
from fastNLP import Trainer
from fastNLP import LossInForward
from torch.optim import Adam
from fastNLP import Tester

from myClue.core import logger  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa

In [8]:
# train_data_bundle_pkl_file = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_data_bundle.pkl'
model_path = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf_bert_embed'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# logger.warn('加载数据集')
# data_bundle = load_serialize_obj(train_data_bundle_pkl_file)
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
bert_embed = BertEmbedding(vocab=char_vocab, model_dir_or_name='/content/drive/My Drive/data/bert-chinese-wwm', requires_grad=False)
# word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name=None, embedding_dim=100)
logger.warn('神经网络模型')
model = BiLSTMCRF(embed=bert_embed, num_classes=len(target_vocab), num_layers=1, hidden_size=200, dropout=0.5,
                  target_vocab=target_vocab)
logger.info(model)
logger.warn('训练超参数设定')
loss = LossInForward()
optimizer = Adam([param for param in model.parameters() if param.requires_grad])
# metric = AccuracyMetric()
metric = SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))

2020-05-21 01:55:02 W [<ipython-input-8-caf39a0c5e2a>:9] 获取词典
2020-05-21 01:55:02 I [<ipython-input-8-caf39a0c5e2a>:11] char_vocab:Vocabulary(['科', '技', '全', '方', '位']...)
2020-05-21 01:55:02 I [<ipython-input-8-caf39a0c5e2a>:13] target_vocab:Vocabulary(['O', 'B-PER.NOM', 'I-PER.NOM', 'B-LOC.NAM', 'I-LOC.NAM']...)
2020-05-21 01:55:02 I [<ipython-input-8-caf39a0c5e2a>:16] 词典序列化:/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf_bert_embed/vocab_char.pkl
2020-05-21 01:55:02 W [<ipython-input-8-caf39a0c5e2a>:17] 选择预训练词向量


loading vocabulary file /content/drive/My Drive/data/bert-chinese-wwm/vocab.txt


2020-05-21 01:55:05 W [<ipython-input-8-caf39a0c5e2a>:20] 神经网络模型
2020-05-21 01:55:05 I [<ipython-input-8-caf39a0c5e2a>:23] BiLSTMCRF(
  (embed): BertEmbedding(
    (dropout_layer): Dropout(p=0, inplace=False)
    (model): _BertWordModel(
      (encoder): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(3405, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768,

Load pre-trained BERT parameters from file /content/drive/My Drive/data/bert-chinese-wwm/chinese_wwm_pytorch.bin.
Start to generate word pieces for word.
87 words are unsegmented. Among them, 51 added to the BPE vocab.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 模型训练
batch_size = 32
n_epochs = 200
early_stopping = 10
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=[EarlyStopCallback(early_stopping)])
logger.warn('开始训练')
trainer.train()

input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 



2020-05-21 01:55:08 W [<ipython-input-10-d54362d8f065>:18] 开始训练


training epochs started 2020-05-21-01-55-08-765985


Evaluate data in 1.83 seconds!


2020-05-21 01:55:25 W [callback.py:38] ======epoch : 1 , early stopping : 0/10======
2020-05-21 01:55:25 I [callback.py:40] metric_key : f, metric_value : 0.004283
2020-05-21 01:55:25 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-org.nam': 0.0, 'pre-org.nam': 0.0, 'rec-org.nam': 0.0, 'f-per.nam': 0.0, 'pre-per.nam': 0.0, 'rec-per.nam': 0.0, 'f-per.nom': 0.008264, 'pre-per.nom': 0.038462, 'rec-per.nom': 0.00463, 'f-loc.nom': 0.0, 'pre-loc.nom': 0.0, 'rec-loc.nom': 0.0, 'f-gpe.nam': 0.0, 'pre-gpe.nam': 0.0, 'rec-gpe.nam': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f': 0.004283, 'pre': 0.028571, 'rec': 0.002315}}


...

2020-05-21 02:12:23 W [callback.py:38] ======epoch : 65 , early stopping : 9/10======
2020-05-21 02:12:23 I [callback.py:40] metric_key : f, metric_value : 0.79661
2020-05-21 02:12:23 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-org.nam': 0.571429, 'pre-org.nam': 0.742857, 'rec-org.nam': 0.464286, 'f-per.nam': 0.765027, 'pre-per.nam': 0.736842, 'rec-per.nam': 0.795455, 'f-per.nom': 0.892523, 'pre-per.nom': 0.900943, 'rec-per.nom': 0.884259, 'f-loc.nom': 0.285714, 'pre-loc.nom': 0.75, 'rec-loc.nom': 0.176471, 'f-gpe.nam': 0.869565, 'pre-gpe.nam': 0.857143, 'rec-gpe.nam': 0.882353, 'f-loc.nam': 0.588235, 'pre-loc.nam': 0.714286, 'rec-loc.nam': 0.5, 'f-org.nom': 0.5, 'pre-org.nom': 0.666667, 'rec-org.nom': 0.4, 'f': 0.79661, 'pre': 0.835025, 'rec': 0.761574}}


Evaluate data in 1.9 seconds!
Evaluation on dev at Epoch 65/200. Step:2795/8600: 
SpanFPreRecMetric: f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-org.nam=0.571429, pre-org.nam=0.742857, rec-org.nam=0.464286, f-per.nam=0.765027, pre-per.nam=0.736842, rec-per.nam=0.795455, f-per.nom=0.892523, pre-per.nom=0.900943, rec-per.nom=0.884259, f-loc.nom=0.285714, pre-loc.nom=0.75, rec-loc.nom=0.176471, f-gpe.nam=0.869565, pre-gpe.nam=0.857143, rec-gpe.nam=0.882353, f-loc.nam=0.588235, pre-loc.nam=0.714286, rec-loc.nam=0.5, f-org.nom=0.5, pre-org.nom=0.666667, rec-org.nom=0.4, f=0.79661, pre=0.835025, rec=0.761574



2020-05-21 02:12:38 W [callback.py:38] ======epoch : 66 , early stopping : 10/10======
2020-05-21 02:12:38 I [callback.py:40] metric_key : f, metric_value : 0.816514
2020-05-21 02:12:38 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-org.nam': 0.685185, 'pre-org.nam': 0.711538, 'rec-org.nam': 0.660714, 'f-per.nam': 0.765957, 'pre-per.nam': 0.72, 'rec-per.nam': 0.818182, 'f-per.nom': 0.882353, 'pre-per.nom': 0.862832, 'rec-per.nom': 0.902778, 'f-loc.nom': 0.461538, 'pre-loc.nom': 0.666667, 'rec-loc.nom': 0.352941, 'f-gpe.nam': 0.927536, 'pre-gpe.nam': 0.914286, 'rec-gpe.nam': 0.941176, 'f-loc.nam': 0.8, 'pre-loc.nam': 0.8, 'rec-loc.nam': 0.8, 'f-org.nom': 0.666667, 'pre-org.nom': 0.75, 'rec-org.nom': 0.6, 'f': 0.816514, 'pre': 0.809091, 'rec': 0.824074}}
2020-05-21 02:12:38 I [callback.py:46] reach early stopping patience, stop training.
2020-05-21 02:12:38 I [callback.py:55] Early Stopping triggered in epoch 66!


Evaluate data in 1.78 seconds!
Reloaded the best model.

In Epoch:55/Step:2365, got best dev performance:
SpanFPreRecMetric: f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-org.nam=0.581395, pre-org.nam=0.833333, rec-org.nam=0.446429, f-per.nam=0.845714, pre-per.nam=0.850575, rec-per.nam=0.840909, f-per.nom=0.885781, pre-per.nom=0.892019, rec-per.nom=0.87963, f-loc.nom=0.363636, pre-loc.nom=0.8, rec-loc.nom=0.235294, f-gpe.nam=0.914286, pre-gpe.nam=0.888889, rec-gpe.nam=0.941176, f-loc.nam=0.705882, pre-loc.nam=0.857143, rec-loc.nam=0.6, f-org.nom=0.181818, pre-org.nom=1.0, rec-org.nom=0.1, f=0.818742, pre=0.875989, rec=0.768519


{'best_epoch': 55,
 'best_eval': {'SpanFPreRecMetric': {'f': 0.818742,
   'f-gpe.nam': 0.914286,
   'f-gpe.nom': 0.0,
   'f-loc.nam': 0.705882,
   'f-loc.nom': 0.363636,
   'f-org.nam': 0.581395,
   'f-org.nom': 0.181818,
   'f-per.nam': 0.845714,
   'f-per.nom': 0.885781,
   'pre': 0.875989,
   'pre-gpe.nam': 0.888889,
   'pre-gpe.nom': 0.0,
   'pre-loc.nam': 0.857143,
   'pre-loc.nom': 0.8,
   'pre-org.nam': 0.833333,
   'pre-org.nom': 1.0,
   'pre-per.nam': 0.850575,
   'pre-per.nom': 0.892019,
   'rec': 0.768519,
   'rec-gpe.nam': 0.941176,
   'rec-gpe.nom': 0.0,
   'rec-loc.nam': 0.6,
   'rec-loc.nom': 0.235294,
   'rec-org.nam': 0.446429,
   'rec-org.nom': 0.1,
   'rec-per.nam': 0.840909,
   'rec-per.nom': 0.87963}},
 'best_step': 2365,
 'seconds': 1050.16}

In [11]:
# 评测
logger.warn('训练后评估')
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
test_result = tester.test()
logger.info('评估结果：\n{}'.format(test_result))

2020-05-21 02:12:38 W [<ipython-input-11-2a8ad2ebd772>:2] 训练后评估


2020-05-21 02:12:40 I [<ipython-input-11-2a8ad2ebd772>:5] 评估结果：
{'SpanFPreRecMetric': {'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-org.nam': 0.333333, 'pre-org.nam': 0.407407, 'rec-org.nam': 0.282051, 'f-per.nam': 0.733333, 'pre-per.nam': 0.793814, 'rec-per.nam': 0.681416, 'f-per.nom': 0.704545, 'pre-per.nom': 0.688889, 'rec-per.nom': 0.72093, 'f-loc.nom': 0.4, 'pre-loc.nom': 0.5, 'rec-loc.nom': 0.333333, 'f-gpe.nam': 0.828283, 'pre-gpe.nam': 0.788462, 'rec-gpe.nam': 0.87234, 'f-loc.nam': 0.413793, 'pre-loc.nam': 0.6, 'rec-loc.nam': 0.315789, 'f-org.nom': 0.521739, 'pre-org.nom': 1.0, 'rec-org.nom': 0.352941, 'f': 0.673367, 'pre': 0.708995, 'rec': 0.641148}}


Evaluate data in 1.65 seconds!
[tester] 
SpanFPreRecMetric: f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-org.nam=0.333333, pre-org.nam=0.407407, rec-org.nam=0.282051, f-per.nam=0.733333, pre-per.nam=0.793814, rec-per.nam=0.681416, f-per.nom=0.704545, pre-per.nom=0.688889, rec-per.nom=0.72093, f-loc.nom=0.4, pre-loc.nom=0.5, rec-loc.nom=0.333333, f-gpe.nam=0.828283, pre-gpe.nam=0.788462, rec-gpe.nam=0.87234, f-loc.nam=0.413793, pre-loc.nam=0.6, rec-loc.nam=0.315789, f-org.nom=0.521739, pre-org.nom=1.0, rec-org.nom=0.352941, f=0.673367, pre=0.708995, rec=0.641148
